In [326]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [327]:
url = 'Data\AB_NYC_2019.csv'
manhattan_url = 'Data\Manhattan_2019.csv'
brooklyn_url = 'Data\Brooklyn_2019.csv'


<>:1: SyntaxWarning: invalid escape sequence '\A'
<>:2: SyntaxWarning: invalid escape sequence '\M'
<>:3: SyntaxWarning: invalid escape sequence '\B'
<>:1: SyntaxWarning: invalid escape sequence '\A'
<>:2: SyntaxWarning: invalid escape sequence '\M'
<>:3: SyntaxWarning: invalid escape sequence '\B'
C:\Users\Santi\AppData\Local\Temp\ipykernel_9056\1942276224.py:1: SyntaxWarning: invalid escape sequence '\A'
  url = 'Data\AB_NYC_2019.csv'
C:\Users\Santi\AppData\Local\Temp\ipykernel_9056\1942276224.py:2: SyntaxWarning: invalid escape sequence '\M'
  manhattan_url = 'Data\Manhattan_2019.csv'
C:\Users\Santi\AppData\Local\Temp\ipykernel_9056\1942276224.py:3: SyntaxWarning: invalid escape sequence '\B'
  brooklyn_url = 'Data\Brooklyn_2019.csv'


In [357]:
df = pd.read_csv(url)
manhattan_df = pd.read_csv(manhattan_url) 
brooklyn_df = pd.read_csv(brooklyn_url)
scaler = StandardScaler()

In [231]:
def get_price_interpretation(value, lower_threshold = 1.5, higher_threshold = 2):
    if abs(value) > lower_threshold and abs(value) < higher_threshold:
        return 'The price is not average'
    elif abs(value) > higher_threshold:
        return 'The price is beyond average'
    else:
        if value < 0 and abs(value) < 0.1:
            return 'Average price'
        elif value > 0 and value < 0.1:
            return 'Average price'
        elif value < 0:
            return 'Lower price average'
        elif value > 0:
            return 'Higher price average'
        else:
            return 'NaN'

In [372]:
def get_z_prices(dataframe):
    analysis_df = dataframe[['id', 'neighbourhood_group','host_name', 'host_id']].copy()
    analysis_df['minimum_price'] = dataframe['price']*dataframe['minimum_nights']
    analysis_df['z_value_prices'] = scaler.fit_transform(analysis_df[['minimum_price']])
    analysis_df['z_value_prices'] = analysis_df['z_value_prices']
    analysis_df = analysis_df[(analysis_df['z_value_prices'] >= -2) & (analysis_df['z_value_prices'] <= 2)]
    analysis_df['z_interpretation'] = analysis_df['z_value_prices'].apply(get_price_interpretation)
    saturation_df = df.groupby(['neighbourhood_group']).agg({'id':'count'}).reset_index()
    saturation_df['market_share'] = (saturation_df['id']/int(saturation_df['id'].sum()))*100
    analysis_df = pd.merge(analysis_df, saturation_df, on=['neighbourhood_group'], how='left')
    return analysis_df

In [292]:
def get_clean_df(dataframe, neighborhood):
    dataframe = dataframe[[dataframe.columns[-2],dataframe.columns[-3]]].copy()
    dataframe['Neighborhood'] = neighborhood
    dataframe = dataframe.loc[dataframe['Sale Price'] != 0]
    dataframe['Sale Price'].dropna()
    dataframe['Sale Price'].value_counts()
    return dataframe

In [316]:
def get_sale_price_interpretation(value, lower_threshold = 1.5, higher_threshold = 2):
    if abs(value) > lower_threshold and abs(value) < higher_threshold:
        return 'The price is not average'
    elif abs(value) > higher_threshold:
        return 'The price is beyond average'
    else:
        if value < 0 and abs(value) < 0.3:
            return 'Average price'
        elif value > 0 and value < 0.3:
            return 'Average price'
        elif value < 0:
            return 'Lower price average'
        elif value > 0:
            return 'Higher price average'
        else:
            return 'NaN'

In [306]:
def get_ny_z_prices(dataframe):
    analysis_df = dataframe
    analysis_df['z_value_prices'] = scaler.fit_transform(analysis_df[['Sale Price']])
    analysis_df['z_value_prices'] = analysis_df['z_value_prices']
    analysis_df = analysis_df[(analysis_df['z_value_prices'] >= -2) & (analysis_df['z_value_prices'] <= 2)]
    analysis_df['z_interpretation'] = analysis_df['z_value_prices'].apply(get_sale_price_interpretation)
    return analysis_df

In [308]:
def get_clean_new_york_df():
    new_york_df = pd.concat([get_clean_df(manhattan_df,'manhattan'),get_clean_df(brooklyn_df,'brooklyn')])
    new_york_df['Neighborhood'].value_counts()
    new_york_df = get_ny_z_prices(new_york_df)
    return new_york_df

In [324]:
def main():
    final_df = get_z_prices(df)
    final_df.to_csv('NYC_Airbnb_2019.csv')
    new_york_2019_df = get_clean_new_york_df()
    new_york_2019_df.to_csv('NYC_Housing_2019.csv')


In [375]:
main()

C:\Users\Santi\AppData\Local\Temp\ipykernel_9056\2329143974.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  analysis_df['z_interpretation'] = analysis_df['z_value_prices'].apply(get_sale_price_interpretation)
